In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

huggingface_api_token = os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")

In [2]:
!pip install langchain chromadb tiktoken pypdf langchain_huggingface langchain-community sentence-transformers langchain_chroma

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [4]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [5]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [6]:
vector_store = Chroma(
    embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [7]:
# add documents
vector_store.add_documents(docs)

['79f9e661-906a-4d01-9f55-85c47d13b95f',
 'cb18f3b4-6b56-4f70-9c01-f447ea048444',
 '4682b792-c854-44ca-b7aa-95ff7fe7e2d0',
 '9c30137e-2e47-4b5e-ae3f-06bed886678c',
 '53a536af-1956-44d9-a40b-c0222bc1c0d2']

In [8]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['79f9e661-906a-4d01-9f55-85c47d13b95f',
  'cb18f3b4-6b56-4f70-9c01-f447ea048444',
  '4682b792-c854-44ca-b7aa-95ff7fe7e2d0',
  '9c30137e-2e47-4b5e-ae3f-06bed886678c',
  '53a536af-1956-44d9-a40b-c0222bc1c0d2'],
 'embeddings': array([[ 0.00994729,  0.06914339, -0.05147116, ..., -0.03543336,
          0.01284811,  0.0124829 ],
        [ 0.00127744,  0.03129851, -0.02375377, ..., -0.00518359,
         -0.03280612,  0.02737716],
        [-0.10265917,  0.02650811,  0.022715  , ..., -0.03359742,
         -0.07984947, -0.0150771 ],
        [ 0.02123394, -0.02468546, -0.04494373, ..., -0.10995814,
          0.0057256 ,  0.09915376],
        [ 0.0187398 ,  0.04382848, -0.04304255, ..., -0.07801621,
         -0.0784068 , -0.00304188]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [9]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='9c30137e-2e47-4b5e-ae3f-06bed886678c', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='cb18f3b4-6b56-4f70-9c01-f447ea048444', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.")]

In [10]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='9c30137e-2e47-4b5e-ae3f-06bed886678c', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9693601727485657),
 (Document(id='cb18f3b4-6b56-4f70-9c01-f447ea048444', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.14934504032135)]

In [11]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='4682b792-c854-44ca-b7aa-95ff7fe7e2d0', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8436005115509033),
 (Document(id='53a536af-1956-44d9-a40b-c0222bc1c0d2', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.890937328338623)]

In [12]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [13]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['79f9e661-906a-4d01-9f55-85c47d13b95f',
  'cb18f3b4-6b56-4f70-9c01-f447ea048444',
  '4682b792-c854-44ca-b7aa-95ff7fe7e2d0',
  '9c30137e-2e47-4b5e-ae3f-06bed886678c',
  '53a536af-1956-44d9-a40b-c0222bc1c0d2'],
 'embeddings': array([[ 0.00994729,  0.06914339, -0.05147116, ..., -0.03543336,
          0.01284811,  0.0124829 ],
        [ 0.00127744,  0.03129851, -0.02375377, ..., -0.00518359,
         -0.03280612,  0.02737716],
        [-0.10265917,  0.02650811,  0.022715  , ..., -0.03359742,
         -0.07984947, -0.0150771 ],
        [ 0.02123394, -0.02468546, -0.04494373, ..., -0.10995814,
          0.0057256 ,  0.09915376],
        [ 0.0187398 ,  0.04382848, -0.04304255, ..., -0.07801621,
         -0.0784068 , -0.00304188]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [14]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [15]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['79f9e661-906a-4d01-9f55-85c47d13b95f',
  'cb18f3b4-6b56-4f70-9c01-f447ea048444',
  '4682b792-c854-44ca-b7aa-95ff7fe7e2d0',
  '9c30137e-2e47-4b5e-ae3f-06bed886678c',
  '53a536af-1956-44d9-a40b-c0222bc1c0d2'],
 'embeddings': array([[ 0.00994729,  0.06914339, -0.05147116, ..., -0.03543336,
          0.01284811,  0.0124829 ],
        [ 0.00127744,  0.03129851, -0.02375377, ..., -0.00518359,
         -0.03280612,  0.02737716],
        [-0.10265917,  0.02650811,  0.022715  , ..., -0.03359742,
         -0.07984947, -0.0150771 ],
        [ 0.02123394, -0.02468546, -0.04494373, ..., -0.10995814,
          0.0057256 ,  0.09915376],
        [ 0.0187398 ,  0.04382848, -0.04304255, ..., -0.07801621,
         -0.0784068 , -0.00304188]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo